In [1]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

In [2]:

aoi = ee.Geometry.Polygon([[16.423269379713, 48.484524861983],
                          [16.423269379713, 48.125585982779],
                          [16.979571008400, 48.125585982779],
                          [16.979571008400, 48.484524861983]])

In [ ]:
#################################
### export Sentinel 2A images ###
#################################

In [16]:
# filter sentinel 2A dataset
sen2_ic = ee.ImageCollection(ee.ImageCollection('COPERNICUS/S2_SR')
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2018-06-16'), ee.Date('2018-06-18'))
                       .sort('GRANULE_ID', False))

In [17]:
imageS2 = ee.Image(sen2_ic.first()).clip(aoi).select('B.*')

In [18]:
# list band names
imageS2.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [24]:
# download via google drive
task = ee.batch.Export.image.toDrive(image=imageS2,
                                     description='sentinel2_20180617_march',
                                     region=aoi,
                                     fileNamePrefix='export_march',
                                     crs='EPSG:32633',
                                     fileFormat='GeoTIFF',
                                     maxPixels=1800000000)
task.start()

In [55]:
task.status()

{'state': 'COMPLETED',
 'description': 'sentinel2_20180617_march',
 'creation_timestamp_ms': 1623319731152,
 'update_timestamp_ms': 1623320883928,
 'start_timestamp_ms': 1623319751698,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/',
  'https://drive.google.com/'],
 'attempt': 1,
 'id': 'RFN3B4JSR2NEXFPADOV7AWO5',
 'name': 'projects/earthengine-legacy/operations/RFN3B4JSR2NEXFPADOV7AWO5'}

In [ ]:
#################################
### export Sentinel 1 GND ###
#################################

In [3]:
# Next, we filter the S1 archive to get an image over the aoi acquired sometime in August, 2020. 
# Any old image will do fine, so we won't bother to specify the orbit number or whether we want
# the ASCENDING or DESCENDING node
imageS1 = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2018-04-17'), ee.Date('2018-04-21')) 
                       .first() 
                       .clip(aoi))
#.sort('GRANULE_ID', False))

In [4]:
imageS1.bandNames().getInfo()

['VV', 'VH', 'angle']

In [5]:
imageS1.getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [4215, 4082],
   'origin': [7737, 12110],
   'crs': 'EPSG:32633',
   'crs_transform': [10, 0, 527792.7758107622, 0, -10, 5493149.891424081]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [4215, 4082],
   'origin': [7737, 12110],
   'crs': 'EPSG:32633',
   'crs_transform': [10, 0, 527792.7758107622, 0, -10, 5493149.891424081]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [4, 4],
   'origin': [11, 4],
   'crs': 'EPSG:32633',
   'crs_transform': [-12950.15147327166,
    -2856.47758146713,
    811134.9369330469,
    1824.3846291070804,
    -20176.168274845928,
    5448220.278710495]}],
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20180420T050133_20180420T050158_021546_025215_0C31',
 'version': 1624115775572816,
 'properties': {'GRD_Post_Processing_start': 1524208469260,
  '

In [6]:
# download via google drive
task = ee.batch.Export.image.toDrive(image=imageS1.select('VH'),
                                     description='sentinel1_20180420_march_descending',
                                     region=aoi,
                                     fileNamePrefix='S1_20180420_march_d',
                                     crs='EPSG:32633',
                                     fileFormat='GeoTIFF',
                                     maxPixels=1800000000)
task.start()

In [7]:
task.status()

{'state': 'READY',
 'description': 'sentinel1_20180420_march_descending',
 'creation_timestamp_ms': 1624117499975,
 'update_timestamp_ms': 1624117499975,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '3VLMTTAGZTJ7YUQ6VQARPZ64',
 'name': 'projects/earthengine-legacy/operations/3VLMTTAGZTJ7YUQ6VQARPZ64'}

In [ ]:
# specify extend
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              16.423269379713,
              48.484524861983
            ],
            [
              16.423269379713,
              48.125585982779
            ],
            [
              16.423269379713,
              48.125585982779
            ],
            [
              16.423269379713,
              48.484524861983
            ],
            [
              16.423269379713,
              48.484524861983
            ]
          ]
        ]
      }
    }
  ]
}

# get coordinates
coords = geoJSON['features'][0]['geometry']['coordinates']
# create ee.Geometry()
aoi = ee.Geometry.Polygon(coords)